# TP2 Causalité et démarche scientifique

On ne peut pas maîtriser les sujets techniques sans pratique, c'est pourquoi les travaux pratiques (TPs) sont une partie importante du cours de sciences des données.

Si vous restez bloqué plus de quelques minutes sur des questions dans une partie non notée d'un TP, demandez de l'aide à un enseignant ou à un camarade de classe (Expliquer les choses est également bénéfique : la meilleure façon de consolider votre connaissance d'un sujet est de l'expliquer). Ne vous contentez pas de juste partager vos réponses sans explication, personne n'en bénéficierait.


#### TP d'aujourd'hui

Dans le TP d'aujourd'hui, vous allez travailler les compétences suivantes :

1. réaliser des simulations aléatoires avec la librairie *numpy.random* de python
2. concevoir une simulation aléatoire pour tester une hypothèse
3. mettre en forme les résultats de la simulation dans un tableau (*DataFrame* de la libraire *pandas*)
4. visualiser ces résultats par la réalisation de graphes appropriés avec la librairie *seaborn*
5. interpréter les résultats
 
<font color='red'>**La dernière partie du TP (partie 2) est à rendre et notée. Le rendu doit obligatoirement se faire par le biais d'amétice avant la fin de la séance de TP.**</font>

# 1. Détection d'un tricheur dans un jeu de dés (durée indicative 1h)

## 1.1 Simulation aléatoire avec *numpy.random* : lancé de dé à six faces (durée indicative 15min)

Pour faire des tirages aléatoires avec python, nous pouvons utiliser la bibliothèque *numpy.random*

In [ ]:
import numpy.random

Nous commençons par créer un objet `rng` avec la méthode `default_rng`. Cet objet va nous permettre de générer des nombres aléatoires.

In [ ]:
rng = numpy.random.default_rng()

Nous pouvons à présent utiliser la méthode `integers` de l''objet `rng` pour simuler le lancé d'un dé à 6 faces en générant un nombre aléatoire entre 1 et 6 avec une distribution uniforme (c'est à dire que chaque résultat possible, 1, 2, 3, 4, 5 ou 6 a autant de chances d'arriver).

In [ ]:
# low is included, high is excluded
rng.integers(low=1, high=7)

Essayez d'évaluer la cellule ci-dessus plusieurs fois pour vérifier que vous n'obtenez pas toujours le même nombre, mais qu'il s'agit toujours d'un nombre entre 1 et 6.

Essayons à présent de vérifier empiriquement que chaque résultat entre 1 et 6 à autant de chance d'arriver. Pour ce faire, commencons par créer une fonction `throw_a_dice` qui simule le lancé de dé

In [ ]:
def throw_a_dice(rng):
    # throw a dice with six faces
    return rng.integers(1, 7)

Appelons maintenant cette fonction un grand nombre de fois (disons 10,000) et stockons les résultats dans la liste `results`

In [ ]:
N_throws = 10000
results = []
for i in range(N_throws):
    results.append(throw_a_dice(rng))
results

Pour vérifier empiriquement que chaque résultat entre 1 et 6 à autant de chance d'arriver, nous pouvons compter le nombre de fois que chaque résultat est obtenu dans notre échantillon. Pour ce faire, une façon pratique est de convertir la liste en une `DataFrame` de la librairie `pandas` et d'utiliser la méthode `value_counts`

In [ ]:
results = pandas.DataFrame(results)
results

In [ ]:
results.value_counts()

On constate que chaque valeur possible du dé arrive à peu près le nombre de fois. Le nombre d'occurrence pour chaque valeur n'est bien sûr pas exactement le même, du fait du caractère aléatoire des tirages, mais vous pouvez vérifier en refaisant plusieurs fois l'expérience, qu'aucune valeur n'a l'air de revenir systématiquement plus souvent que les autres.

## 1.2 Un tricheur (durée indicative : 10min)

Supposons à présent que dans le cadre d'un jeu de dé, un joueur triche: au lieu de rapporter le résultat d'un lancé de dé, il lance deux dés et rapporte le maximum des résultats obtenus. Par exemple, s'il obtient 2 et 4 il dit qu'il a obtenu 4. S'il obtient 1 et 3, il dit qu'il a obtenu trois, etc.

Nous pouvons simuler le comportement de ce joueur par le biais de la fonction suivante :

In [ ]:
def fake_throw(rng):
    result_dice1 = throw_a_dice(rng)
    result_dice2 = throw_a_dice(rng)
    if result_dice1 > result_dice2:
        return result_dice1
    else:
        return result_dice2

Testez la fonction

In [ ]:
# votre code

Supposons que le jeu auquel on joue est la version suivante du *cul de chouette* (jeu du pays de Galle) : chaque joueur lance 379 dés à six faces et rapporter la somme des résultats obtenus. Le joueur avec le plus grand total gagne la partie.

Ecrivons une fonction pour simuler le résultat que pourrait obtenir notre tricheur dans le cadre de ce jeu :

In [ ]:
def fake_sum_of_379_dices(rng):
    s = 0
    for i in range(379):
        s = s + fake_throw(rng)
    return s

In [ ]:
cheater_result = fake_sum_of_379_dices(rng)
cheater_result

Imaginons que le tricheur nous rapporte ce résultat. Pouvons-nous détecter immédiatement qu'il s'agit d'un tricheur ?

# 1.3 Test d'une hypothèse où comment détecter un tricheur (durée indicative 35 min)

Pour savoir si le résultat rapporté par le tricheur est plausible, nous pouvons le comparer à la distribution des résultats qu'obtiennent des joueurs honnêtes et mesurer la probabilité qu'un joueur honnête obtienne un résultat supérieur ou égal à celui rapporté par le tricheur. Si cette probabilité est extrèmement faible, nous aurons de bonnes raisons de suspecter le tricheur de ne pas être honnête.

Comme nous l'avons vu en cours, nous pouvons utiliser des simulations pour obtenir une approximation de la distribution des résultats qu'obtiennent des joueurs honnêtes.

Commencons par coder une fonction qui génère le résultat obtenu par un joueur honnête (cette fonction ré-utilise la fonction `throw_a_dice` que nous avons créée précédemment :

In [ ]:
def sum_of_379_dices(rng):
    s = 0
    for i in range(379):
        s = s + throw_a_dice(rng)
    return s

Nous pouvons à présent utiliser cette fonction pour simuler un grand nombre de partie pour des joueurs honnêtes (disons 10,000 parties)

In [ ]:
nb_runs = 10000

legit_sums = []
for run_id in range(nb_runs):
    legit_sums.append(sum_of_379_dices(rng))
legit_sums

Pour analyser les résultats de notre simulation plus facilement, stockons-les dans un tableau (`pandas.DataFrame`)

In [ ]:
import pandas
legit_data = pandas.DataFrame({'sum' : legit_sums})
legit_data

Nous pouvons à présent utiliser la libraire *seaborn* pour représenter la distribution des résultats obtenus par simulation sous la forme d'un histogramme

In [ ]:
import seaborn

g = seaborn.displot(data=legit_data, kind='hist', stat='density', bins=25)
g.set(title="Distribution empirique des sommes obtenues par un joueur honnête")

Nous pouvons ajouter sur notre graphique une droite en rouge, indiquant la position de la valeur rapportée par notre tricheur, en utilisant la fonction `axvline` de la libraire `matplotlib.pyplot` une librairie très utilisée pour réaliser des tracés en python (`seaborn` s'appuye sur cette librairie et on utilise souvent `matplotlib.pyplot` pour raffiner/augmenter des tracés obtenus avec `seaborn`).

In [ ]:
import seaborn
import matplotlib.pyplot as plt

g = seaborn.displot(data=legit_data, kind='hist', stat='density', bins=25, aspect=1.5)
plt.axvline(x=cheater_result, color='r')

Comme on peut le constater sur la figure ci-dessous, la valeur rapportée par le tricheur est bien au-delà de toutes valeurs obtenues par un joueur honnête à travers 10,000 simulations. On en déduit que la probabilité d'obtenir une valeur au-moins aussi grande que celle rapportée par le tricheur (*p-value*) est inférieure à $1/10000$.

Sur la base de cette analyse, il semble donc raisonnable de suspecter notre tricheur!

# 2. Généralisation à un nouvel exemple <font color='red'>(partie notée, à rendre individuellement, durée indicative 1h)</font>

<font color='red'>**Cette partie est à rendre et notée. Le rendu doit obligatoirement se faire par le biais d'amétice avant la fin de la séance de TP.**</font> Chacun d'entre vous doit rendre un travail personnel pour cette partie, différent de celui de tous les autres étudiants. Les recopies et plagiats seront séverement sanctionnés.

Dans cette partie, vous devez imaginer et décrire une situation différente de la situation considérée en partie 1, identifier une hypothèse à tester dans le cadre de cette nouvelle situation, proposer une simulation pour tester votre hypothèse, réaliser cette simulation et interpréter les résultats.

Vous pouvez proposer des modifications très simples (par exemple, modifier le nombre de faces des dés considérés, le nombre de jets de dés réalisés, ou la manière dont le tricheur triche) ou plus compliquées (par exemple, envisager un jeu prenant une forme complètement différente). Vous pouvez gagner des points en proposant des modifications plus ambitieuses, mais il est tout à fait possible d'obtenir une très bonne note avec une modification très simple, **à condition de montrer une compréhension détaillée de l'impact de cette modification sur les résultats obtenus** (cf. grille d'évaluation ci-dessous).


La grille d'évaluation suivante sera utilisée pour noter vos travaux:

1. Motivation de l’analyse réalisée
    - Analyse clairement pertinente et bien justifiée : 5
    - Motivation partiellement claire, mais manque de précision : 2,5
    - Motivation incompréhensible ou absente : 0

2. Interprétation des résultats
    - Interprétation correcte et pertinente des résultats : 5
	- Interprétation partiellement correcte, mais quelques lacunes : 2,5
	- Interprétation douteuse, incomplète ou manquante : 0

3. Capacité à généraliser
    - Situation simulée très différente de celle considérée en partie 1 : 5
    - Situation simulée présentant des différences subtantielles avec celle de la partie 1 : 2,5
    - Simulation d'une situation quasi identique à celle de la partie 1 : 0

4. Correction du code
    - Code fonctionnel et sans erreurs : 2,5
    - Code fonctionnel mais avec des erreurs partielles : 1
    - Code non fonctionnel ou contenant de nombreuses erreurs : 0

5. Propreté du code
    - Code propre, bien structuré et lisible : 2,5
    - Code lisible mais désordonné ou manquant de clarté : 1
    - Code difficilement lisible et mal structuré : 0

Note totale : /20


## 2.1 Commencez par décrirer la nouvelle situation considérée

Votre réponse

## 2.2 Décrivez l'hypothèse que vous souhaitez tester

Votre réponse

## 2.3 Décrivez la simulation que vous souhaitez réaliser pour tester votre hypothèse

Votre réponse

## 2.4 Implémentez la simulation proposée

In [ ]:
# Votre réponse

## 2.5 Représentez graphiquement les résultats

In [ ]:
# Votre réponse

## 2.6 Interprétation des résultats

Expliquez notamment en quoi, et pourquoi, les résultats sont différents de ceux obtenus dans la partie 1

Votre réponse

# Crédits

Ce cours est inspiré du cours data8 donné à UC Berkeley et en ré-utilise avec certaines modifications une partie des matériels (ces matériels sont généreusement mis à disposition publiquement sous licence Creative Commons avec attribution, consultez [https://www.data8.org](https://www.data8.org) pour plus d'informations.